In [1]:
import time
import numpy as np
from GEECSPythonAPI import GEECSDevice
from GEECSPythonAPI import OptimizationControl
import yaml

In [2]:
# Axicon Horizontal vertical looking at minimizing STD 
# yamlfile="AxiconHorizontalVertical.yaml"
# evaluator_str = "-abs(np.std(outs))"
# TestPoints_dict = {"AxiconHorizontal":0.5,"AxiconVertical":0.5}
# InitialPoints_dict = {"AxiconHorizontal":[0.85,0.1],"AxiconVertical":[0.85,0.1]}
#tolerance=0.001

#hpd focal scan
yamlfile="PWfocusLpMode.yaml"
evaluator_str = "np.median(outs)"
TestPoints_dict = {"exposure":0.0005}
InitialPoints_dict = {"Position":[0.7,0.3]}
tolerance=0.01

# yamlfile="AxiconHorizontalVertical.yaml"
# evaluator_str = "-abs(np.std(outs))"
# TestPoints_dict = {"AxiconHorizontal":0.5,"AxiconVertical":0.5}
# InitialPoints_dict = {"AxiconHorizontal":[0.85,0.1],"AxiconVertical":[0.85,0.1]}
#tolerance=0.01


# this one is good for aligning to a crosshair
#    return {'f': -abs(np.median(outs)-672)}
#This is for minimizing fluctuation of the data
#    return {'f': eval(evaluator_str)}

In [3]:
#define a "yaml file" with no entries for variables. These will be filled in used the "bella_config" yamml file
YAML = """
xopt:
    dump_file: dump.yaml
generator:
    name: upper_confidence_bound
    n_initial: 5
    optim:
        num_restarts: 1
    acq:
        proximal_lengthscales: []

evaluator:
    function: __main__.geecs_measurement

vocs:
    variables:
        {}
    objectives: {f: "MAXIMIZE"}

"""
yaml_output = yaml.safe_load(YAML)

with open(yamlfile, "r") as stream:
    try:
        bella_config=yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)
bella_config

{'control_variables': {'exposure': {'device_name': 'CAM-2BL-ProbeM6M7',
   'device_variable': 'exposure',
   'range': [3e-05, 0.001],
   'ipaddress': '0.0.0.0',
   'port': 9999,
   'busy': 0,
   'newDataFlag': 0,
   'tcp_client': 'na',
   'actual_value': 0.0,
   'set': True,
   'set_value': 0.0,
   'proximal_lengthscale': 0.001,
   'objective': False},
  'Charge': {'device_name': 'CAM-PL1-LPMode',
   'device_variable': 'MaxCounts',
   'ipaddress': '0.0.0.0',
   'port': 9999,
   'busy': 0,
   'newDataFlag': 0,
   'tcp_client': 'na',
   'actual_value': 0.0,
   'method': 'MAXIMIZE',
   'objective': True,
   'set': False}},
 'objective_variables': {'Charge': {'device_name': 'CAM-2BL-ProbeM6M7',
   'device_variable': 'MaxCounts',
   'range': [0, 1000.0],
   'ipaddress': '0.0.0.0',
   'port': 9999,
   'busy': 0,
   'newDataFlag': 0,
   'tcp_client': 'na',
   'actual_value': 0.0,
   'method': 'MAXIMIZE'}},
 'database': {'name': 'loasis',
  'port': 3306,
  'ipaddress': '192.168.13.23',
  'user

In [4]:
optimization_controls=[]
ranges=[]
obj_tags=[]
normalize_ranges=False
cv_tags=dict(bella_config['control_variables'])
optimization_objects_dict=dict()
optimization_function_dict=dict()
for cv_tag, cv_config in cv_tags.items():
    if cv_config['set']:
        #find the variables in the config yaml file chosen as select and create a control object for them
        opt_contr=OptimizationControl(device_name=cv_config['device_name'],variable=cv_config['device_variable'],bounds=cv_config['range'])
        opt_contr.device_initialize()
        #optimization_controls.append(opt_contr)
        
        
        

        
        #create a dictionary with the keys that matech what are put into the VOCS structure that map to the 
        #xopt control objects so that they can be called later in geecs_measurement
        optimization_objects_dict[cv_tag]=opt_contr
        
        
        #calculate normalized ranges
        ranges.append(cv_config['range'])
        range_offset=[0,cv_config['range'][1]-cv_config['range'][0]]
        range_norm=[0,1]
        print(range_offset)
        
        #update the VOCS structure to use values taken from the config file
        if normalize_ranges:
            yaml_output['vocs']['variables'][cv_tag]=[0,1]
            yaml_output['generator']['acq']['proximal_lengthscales'].append(cv_config['proximal_lengthscale']/range_offset[1])
        else:
            yaml_output['vocs']['variables'][cv_tag]=cv_config['range']
    
    
    if cv_config['objective']:
        opt_contr=OptimizationControl(device_name=cv_config['device_name'],variable=cv_config['device_variable'])
        optimization_function_dict[cv_tag]=opt_contr
        opt_contr.device_initialize()
        

[0, 0.00097]


In [5]:
outputs=obj_tags

def geecs_measurement(input_dict):

    for i in list(input_dict.keys()):
        set_val=float(input_dict[i])
        print('set '+str(i)+' to '+str(set_val))
        optimization_objects_dict[i].xopt_set(float(set_val))

    outs=[]
    counter=0
    while counter<bella_config['optimization_parameters']['shots_per_step']:
        for i in outputs:
            val=float(optimization_function_dict[i].xopt_get())
            outs.append(val)
        counter=counter+1
    print(outs)

###############
## TO DO
###############
## Add some data processing on the resulting on the acquired data.
## for example, remove the extreme values, then maybe take a median
## also need to test with passing all acquired data points rather than some stat

# this one is good for aligning to a crosshair
#    return {'f': -abs(np.median(outs)-672)}
#This is for minimizing fluctuation of the data
#    return {'f': -abs(np.std(outs))}
    return {'f': eval(evaluator_str)}

In [6]:
geecs_measurement(TestPoints_dict)

set exposure to 0.0005
can set
set command accepted


timeout: timed out

In [ ]:
optimization_function_dict["Charge"].xopt_get()